In [33]:
import glob
import re
import sqlite3
import time
import traceback
from pathlib import Path

import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
from PIL import Image

In [8]:
# Paths
mapping_path = r"C:\Users\shubh\Downloads\MSc 3rd Sem\Galaxy_Data\gz2_filename_mapping.csv"
labels_path = r"C:\Users\shubh\Downloads\MSc 3rd Sem\Galaxy_Data\gz2_hart16.csv"

# Load
df_map = pd.read_csv(mapping_path, dtype={"objid": str, "asset_id": str})
df_lbl = pd.read_csv(labels_path, dtype={"dr7objid": str})

print("mapping shape:", df_map.shape)
print("labels  shape:", df_lbl.shape)

print("Mapping file:")
print(df_map.head())

print("\nLabels file:")
print(df_lbl.head())


mapping shape: (355990, 3)
labels  shape: (239695, 231)
Mapping file:
                objid    sample asset_id
0  587722981736120347  original        1
1  587722981736579107  original        2
2  587722981741363294  original        3
3  587722981741363323  original        4
4  587722981741559888  original        5

Labels file:
             dr7objid          ra        dec     rastring    decstring  \
0  587732591714893851  179.042984  60.522518  11:56:10.32  +60:31:21.1   
1  588009368545984617  135.084396  52.494240  09:00:20.26  +52:29:39.3   
2  587732484359913515  183.371979  50.741508  12:13:29.27  +50:44:29.4   
3  587741723357282317  186.251953  28.558598  12:25:00.47  +28:33:31.0   
4  587738410866966577  161.086395  14.084465  10:44:20.73  +14:05:04.1   

     sample gz2_class  total_classifications  total_votes  \
0  original      Sc+t                     45          342   
1  original      Sb+t                     42          332   
2  original        Ei                     

In [9]:
print("\nmapping columns:", df_map.columns.tolist())
print("\nlabels  columns (first 20):", df_lbl.columns.tolist()[:20])

# quick peek
display(df_map.head())
display(df_lbl.head())


mapping columns: ['objid', 'sample', 'asset_id']

labels  columns (first 20): ['dr7objid', 'ra', 'dec', 'rastring', 'decstring', 'sample', 'gz2_class', 'total_classifications', 'total_votes', 't01_smooth_or_features_a01_smooth_count', 't01_smooth_or_features_a01_smooth_weight', 't01_smooth_or_features_a01_smooth_fraction', 't01_smooth_or_features_a01_smooth_weighted_fraction', 't01_smooth_or_features_a01_smooth_debiased', 't01_smooth_or_features_a01_smooth_flag', 't01_smooth_or_features_a02_features_or_disk_count', 't01_smooth_or_features_a02_features_or_disk_weight', 't01_smooth_or_features_a02_features_or_disk_fraction', 't01_smooth_or_features_a02_features_or_disk_weighted_fraction', 't01_smooth_or_features_a02_features_or_disk_debiased']


,objid,sample,asset_id
0,587722981736120347,original,1
1,587722981736579107,original,2
2,587722981741363294,original,3
3,587722981741363323,original,4
4,587722981741559888,original,5


,dr7objid,ra,dec,rastring,decstring,sample,gz2_class,total_classifications,total_votes,t01_smooth_or_features_a01_smooth_count,...,t11_arms_number_a36_more_than_4_fraction,t11_arms_number_a36_more_than_4_weighted_fraction,t11_arms_number_a36_more_than_4_debiased,t11_arms_number_a36_more_than_4_flag,t11_arms_number_a37_cant_tell_count,t11_arms_number_a37_cant_tell_weight,t11_arms_number_a37_cant_tell_fraction,t11_arms_number_a37_cant_tell_weighted_fraction,t11_arms_number_a37_cant_tell_debiased,t11_arms_number_a37_cant_tell_flag
0,587732591714893851,179.042984,60.522518,11:56:10.32,+60:31:21.1,original,Sc+t,45,342,0,...,0.450,0.450,0.482646,0,16,16.0,0.400,0.400,0.394506,0
1,588009368545984617,135.084396,52.494240,09:00:20.26,+52:29:39.3,original,Sb+t,42,332,1,...,0.512,0.503,0.504833,0,13,13.0,0.317,0.323,0.322743,0
2,587732484359913515,183.371979,50.741508,12:13:29.27,+50:44:29.4,original,Ei,36,125,28,...,0.000,0.000,0.000000,0,0,0.0,0.000,0.000,0.000000,0
3,587741723357282317,186.251953,28.558598,12:25:00.47,+28:33:31.0,original,Sc+t,28,218,1,...,0.240,0.240,0.241322,0,6,6.0,0.240,0.240,0.239765,0
4,587738410866966577,161.086395,14.084465,10:44:20.73,+14:05:04.1,original,Er,43,151,33,...,0.000,0.000,0.000000,0,0,0.0,0.000,0.000,0.000000,0


In [10]:
images_dir   = r"C:\Users\shubh\Downloads\MSc 3rd Sem\Galaxy_Data\images"

In [11]:
out_merged_csv = r"C:\Users\shubh\Downloads\MSc 3rd Sem\Galaxy_Morphology_Final\merged_labels_assets.csv"

In [12]:
df = pd.merge(df_map, df_lbl, left_on="objid", right_on="dr7objid", how="inner", suffixes=("_map", "_lbl"))

print("merged shape:", df.shape)
display(df.head())
display(df[["asset_id", "objid", "dr7objid", "gz2_class"]].head())

merged shape: (239695, 234)


,objid,sample_map,asset_id,dr7objid,ra,dec,rastring,decstring,sample_lbl,gz2_class,...,t11_arms_number_a36_more_than_4_fraction,t11_arms_number_a36_more_than_4_weighted_fraction,t11_arms_number_a36_more_than_4_debiased,t11_arms_number_a36_more_than_4_flag,t11_arms_number_a37_cant_tell_count,t11_arms_number_a37_cant_tell_weight,t11_arms_number_a37_cant_tell_fraction,t11_arms_number_a37_cant_tell_weighted_fraction,t11_arms_number_a37_cant_tell_debiased,t11_arms_number_a37_cant_tell_flag
0,587722981741363294,original,3,587722981741363294,182.925262,-1.092357,12:11:42.06,-01:05:32.5,original,Sb,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0
1,587722981741363323,original,4,587722981741363323,182.970108,-1.219537,12:11:52.83,-01:13:10.3,original,Sc?l,...,0.0,0.0,0.0,0,1,1.0,1.0,1.0,1.0,1
2,587722981741559888,original,5,587722981741559888,183.438095,-1.238414,12:13:45.14,-01:14:18.3,original,Er,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0
3,587722981741625481,original,6,587722981741625481,183.473999,-1.231429,12:13:53.76,-01:13:53.1,original,Sc1t,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0
4,587722981741625484,original,7,587722981741625484,183.477783,-1.084604,12:13:54.67,-01:05:04.6,original,Sb,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0


,asset_id,objid,dr7objid,gz2_class
0,3,587722981741363294,587722981741363294,Sb
1,4,587722981741363323,587722981741363323,Sc?l
2,5,587722981741559888,587722981741559888,Er
3,6,587722981741625481,587722981741625481,Sc1t
4,7,587722981741625484,587722981741625484,Sb


In [13]:
df["filename"] = df["asset_id"].astype(str) + ".jpg"
df["file_path"] = df["filename"].apply(lambda f: os.path.join(images_dir, f))
df["file_exists"] = df["file_path"].apply(os.path.exists)

print("Files matched:", df["file_exists"].sum(), " / ", len(df))
print("Missing files:", len(df) - df["file_exists"].sum())

# show a few missing examples (if any)
display(df.loc[~df["file_exists"], ["asset_id", "filename", "objid"]].head())

Files matched: 239573  /  239695
Missing files: 122


,asset_id,filename,objid
341,374,374.jpg,587722982290620479
5212,7349,7349.jpg,587725471205490852
5213,7350,7350.jpg,587725471205556327
5214,7351,7351.jpg,587725471205556424
8380,10959,10959.jpg,587725577498591551


In [14]:
df = df[df["file_exists"]].copy()
print("After filtering:", len(df))
df.shape

After filtering: 239573


(239573, 237)

In [15]:
df.to_csv(out_merged_csv, index=False)

Now, we need to finalize the target columns, extract the 37 debiased probability columns (make sure they sum to 1 within each task, but not necessarily across all 37 since some tasks are conditional) and save them in a NumPy array (shape: [num_samples, 37]).

In [16]:
target_cols = [col for col in df.columns if col.endswith("_debiased")]
print(len(target_cols))   
print(target_cols[:10])   


37
['t01_smooth_or_features_a01_smooth_debiased', 't01_smooth_or_features_a02_features_or_disk_debiased', 't01_smooth_or_features_a03_star_or_artifact_debiased', 't02_edgeon_a04_yes_debiased', 't02_edgeon_a05_no_debiased', 't03_bar_a06_bar_debiased', 't03_bar_a07_no_bar_debiased', 't04_spiral_a08_spiral_debiased', 't04_spiral_a09_no_spiral_debiased', 't05_bulge_prominence_a10_no_bulge_debiased']


In [17]:
Y = df[target_cols].to_numpy()   
print(Y.shape)  


(239573, 37)


In [18]:
# List of tasks and their debiased columns
tasks = {
    "t01_smooth_or_features": [
        "t01_smooth_or_features_a01_smooth_debiased",
        "t01_smooth_or_features_a02_features_or_disk_debiased",
        "t01_smooth_or_features_a03_star_or_artifact_debiased"
    ],
    "t02_edgeon": [
        "t02_edgeon_a04_yes_debiased",
        "t02_edgeon_a05_no_debiased"
    ],
    "t03_bar": [
        "t03_bar_a06_bar_debiased",
        "t03_bar_a07_no_bar_debiased"
    ],
    "t04_spiral": [
        "t04_spiral_a08_spiral_debiased",
        "t04_spiral_a09_no_spiral_debiased"
    ],
    "t05_bulge_prominence": [
        "t05_bulge_prominence_a10_no_bulge_debiased",
        "t05_bulge_prominence_a11_just_noticeable_debiased",
        "t05_bulge_prominence_a12_obvious_debiased",
        "t05_bulge_prominence_a13_dominant_debiased"
    ],
    "t06_odd": [
        "t06_odd_a14_yes_debiased",
        "t06_odd_a15_no_debiased"
    ],
    "t07_rounded": [
        "t07_rounded_a16_completely_round_debiased",
        "t07_rounded_a17_in_between_debiased",
        "t07_rounded_a18_cigar_shaped_debiased"
    ],
    "t08_odd_feature": [
        "t08_odd_feature_a19_ring_debiased",
        "t08_odd_feature_a20_lens_or_arc_debiased",
        "t08_odd_feature_a21_disturbed_debiased",
        "t08_odd_feature_a22_irregular_debiased",
        "t08_odd_feature_a23_other_debiased",
        "t08_odd_feature_a24_merger_debiased"
    ],
    "t09_bulge_shape": [
        "t09_bulge_shape_a25_round_debiased",
        "t09_bulge_shape_a26_boxy_debiased",
        "t09_bulge_shape_a27_no_bulge_debiased"
    ],
    "t10_spiral_winding": [
        "t10_spiral_winding_a28_tight_debiased",
        "t10_spiral_winding_a29_medium_debiased",
        "t10_spiral_winding_a30_loose_debiased"
    ],
    "t11_spiral_arm_count": [
        "t11_spiral_arm_count_a31_1_debiased",
        "t11_spiral_arm_count_a32_2_debiased",
        "t11_spiral_arm_count_a33_3_debiased",
        "t11_spiral_arm_count_a34_4_debiased",
        "t11_spiral_arm_count_a36_more_than_4_debiased",
        "t11_spiral_arm_count_a37_cant_tell_debiased"
    ]
}


In [21]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit


In [23]:
# Step 0: clean Y (replace NaNs with 0)
Y_clean = np.nan_to_num(Y, nan=0.0)

# Step 1: create stratified splitter
msss_outer = MultilabelStratifiedShuffleSplit(
    n_splits=1, 
    test_size=15000, 
    random_state=42
)

# Step 2: run stratified split
train_full_idx, subset_idx = next(msss_outer.split(np.zeros(len(Y_clean)), Y_clean))

print(len(subset_idx))  # should be 15000
print(len(train_full_idx))  # the rest


15089
224484


In [24]:
# Step 3: Stratified split of the 15,089 subset into 10k / 5k
subset_Y = Y_clean[subset_idx]

msss_inner = MultilabelStratifiedShuffleSplit(
    n_splits=1,
    test_size=5000,   # we want 5k validation
    random_state=42
)

train_idx, val_idx = next(msss_inner.split(np.zeros(len(subset_Y)), subset_Y))

# map indices back to original dataset
subset_train_idx = subset_idx[train_idx]
subset_val_idx   = subset_idx[val_idx]

print(len(subset_train_idx))  # should be ~10089
print(len(subset_val_idx))    # should be 5000


10076
5013


In [32]:
print(subset_idx.shape)
print(subset_Y.shape)

(15089,)
(15089, 37)


In [35]:
# Parameters
img_size = (128, 128)
# subset_train_idx and subset_val_idx were obtained from your stratified splitting
all_required_idx = np.concatenate([subset_train_idx, subset_val_idx])

# List all image filenames (sorted so indices match)
image_files = sorted([os.path.join(images_dir, f) 
                      for f in os.listdir(images_dir) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

# Prepare arrays
X_train = []
X_val = []

for i, idx in enumerate(all_required_idx):
    path = image_files[idx]  # index in the full dataset
    img = Image.open(path).convert('L')  # grayscale
    img = img.resize(img_size)
    img_array = np.array(img, dtype=np.float32) / 255.0  # normalize to [0,1]

    if idx in subset_train_idx:
        X_train.append(img_array)
    else:
        X_val.append(img_array)

# Convert lists to numpy arrays with shape (num_samples, 128, 128, 1)
X_train = np.expand_dims(np.array(X_train), -1)
X_val = np.expand_dims(np.array(X_val), -1)

In [36]:
# Save arrays
np.save("X_train.npy", X_train)
np.save("X_val.npy", X_val)
np.save("Y_train.npy", Y_clean[subset_train_idx])
np.save("Y_val.npy", Y_clean[subset_val_idx])

print("Shapes:")
print("X_train:", X_train.shape)
print("X_val:", X_val.shape)
print("Y_train:", Y_clean[subset_train_idx].shape)
print("Y_val:", Y_clean[subset_val_idx].shape)

Shapes:
X_train: (10076, 128, 128, 1)
X_val: (5013, 128, 128, 1)
Y_train: (10076, 37)
Y_val: (5013, 37)
